# 3D_HOUSES

## Import all libraries

In [ ]:
# Code/system libraries
from glob import glob
from typing import Tuple, List
import natsort

# Data libraries
import numpy as np
import pandas as pd

# Visualization libraries
import ipywidgets as widgets
from mayavi import mlab
import plotly.graph_objects as go
import plotly.io as pio 

# API libraries
import json
import requests

# GeoData/rasters libraries
import geopandas as gpd
import rasterio
import rioxarray as rxr
from shapely.geometry import Point, Polygon
from shapely.ops import cascaded_union

## Global Variables

In [ ]:
# Specify the paths to DSM and DTM folders + paths to .shp files

DSM_path = "./DSM/**/*.tif"
DTM_path =  "./DTM/**/*.tif"

shp_paths = ["./Bpn_CaBu.shp", "./Bpn_ReBu.shp"]

property = {}

## Create a dictionary of GTiff's paths and bounds

In [ ]:
def load_data(DSM_path:str, DTM_path:str) -> dict:
    '''
    This function creates a dictionary with two sorted lists for DSM/DTM files
    and one similarly sorted list with their bounds in L72 Lambert coordinates.
    '''
    
    data = {}
    # natsorted deals with 1-10 ordering
    data['DSM_list'] = natsort.natsorted([file for file in glob(DSM_path, recursive=True)])
    data['DTM_list'] = natsort.natsorted([file for file in glob(DTM_path, recursive=True)])
    data['bounds'] = []
    
    # bounds in DTM and DSM are equivalent
    for path in data['DSM_list']:
        with rasterio.open(path, driver="GTiff") as tif:
            data['bounds'].append(np.array(tif.bounds))

    return data

## Fetch the L_72 coordinates from the API with address

In [ ]:
def fetch_coord(address:str)  -> Point:
    '''
    This function fetches the L72 coordinates of a given address
    from the loc.geopunt.be API.
    '''
    
    response = requests.get(f"https://loc.geopunt.be/v4/location?q={address}")
    data = json.loads(response.content)
    
    if not data['LocationResult']:
        raise ValueError(f"The given address ({address}) could not be found by the API")
    
    long_x_lambert72 = data['LocationResult'][0]['Location']['X_Lambert72']
    lat_y_lambert72 = data['LocationResult'][0]['Location']['Y_Lambert72']
    
    return Point(float(long_x_lambert72),float(lat_y_lambert72))

## Find the index of the .tif files containing the coordinates

In [ ]:
def find_tif(data:dict, coord:Point) -> int:
    '''
    This function checks which DSM/DTM files contain the Point
    coordinates by checking against the bounds, and returns the
    index if a DSM/DTM is found.
    '''
    
    # unpacking the x,y from Point
    x, y = coord.xy
    
    for tif_index, bounds in enumerate(data['bounds']):
        x_min, y_min, x_max, y_max = bounds
        if (x_min < x < x_max) and (y_min < y < y_max):
            return tif_index
        
    raise ValueError(f"No DSM/DTM containing the given coordinates ({coord.xy}) could be found")

## Find the polygon containing the coordinates

In [ ]:
def find_polygon(coord:Point, shp_paths:List[str]) -> Polygon:
    '''
    This function searches for the polygon(s) containing the Point coordinates
    within a list of .shp files containing cadastrial information. If multiple
    polygons are found, they are joined together to form a single polygon.
    '''

    for path in shp_paths:
        
        # Ultra-efficient search for a polygon intersecting the mask object
        polys = gpd.read_file(path, mask=coord).geometry
        
        if not polys.empty:
            polys_list = [poly for poly in gpd.read_file(path, mask=coord).geometry]
            poly = cascaded_union(polys)
            return poly
        
    raise ValueError(f"No polygon containing the given coordinates ({coord.xy}) could be found")

## Crop DSM & DTM

In [ ]:
def crop_tif(data, tif_index, poly, shape_cut=True) -> Tuple[np.ndarray]:
    '''
    This function clips the DSM and DTM files with the shape or the bounds
    of the polygon. If clipping with the shape, nan.values are set to zero
    to later have better rendering of the walls (avoiding overclipping).
    '''
    
    DSM = rxr.open_rasterio(data['DSM_list'][tif_index],masked=True)
    DTM = rxr.open_rasterio(data['DTM_list'][tif_index],masked=True)
    
    # First clipping with bounds optimizes the processing
    left, bottom, right, top = poly.bounds
    DSM_clip = DSM.rio.clip_box(left, bottom, right, top)
    DTM_clip = DTM.rio.clip_box(left, bottom, right, top)

    if shape_cut:
        DSM_clip = DSM_clip.rio.clip([poly.__geo_interface__])
        DSM_clip = np.nan_to_num(DSM_clip, nan=0)

        DTM_clip = DTM_clip.rio.clip([poly.__geo_interface__])
        DTM_clip = np.nan_to_num(DTM_clip, nan=0)
    
    # Close the .tif files to avoid memory leaks
    DSM.close()
    DTM.close()
    
    return DSM_clip, DTM_clip

## Generate CHM

In [ ]:
def CHMer(DSM_clip, DTM_clip) -> np.ndarray:
    '''
    This function generates a Canopy Height Models (CHM) that renders terrain
    features from the ground up.
    '''
    return DSM_clip - DTM_clip

## 3D Rendering (plotly)

In [ ]:
def render_3D_2(CHM_clip: np.ndarray):
    '''
    ©Harold, thanks for this function ! 
    This function uses plotly to create an interactive 3D plot annotated 
    with real-estate information estimated from the polygon and the CHM. 
    '''
    # Allows the rendering to be displayed in the browser
    pio.renderers.default='browser' 
    
    # Retrieves the NxM array from the BxNxM xarray object
    arr = CHM_clip.squeeze().data
    # Pads the array for better rendering
    arr = np.pad(arr, [(5, ), (5, )], mode='constant')
    
    # Take the length to have proper ratio in the rendering
    N = len(arr[:,0])
    M = len(arr[0,:])
    
    clipped_df = pd.DataFrame(arr)

    fig = go.Figure(data=[go.Surface(z=clipped_df, colorscale='Portland')], 
                    layout=go.Layout(
                        annotations=[
                            go.layout.Annotation(
                                text=f"""
                                Max_height: {property["Max_height"]} m<br>
                                Estimated surface area : {property['Estimated ground living area']} m2<br>
                                Estimated Max number of floors : {property['Estimated Max number of floors']}<br>
                                """,
                                align='left',
                                showarrow=False,
                                xref='paper',
                                yref='paper',
                                x=0.1,
                                y=1)
                    ]
            )
    )
    
    fig.update_traces(contours_z=dict(show=True, usecolormap=True,
                                      highlightcolor="turquoise", project_z=True)
    )
    
    fig.update_layout(title=(dropdown_address.value), 
                      scene = {"aspectratio": {"x": (N/N), "y":(N/M), "z": 0.5}},
                      autosize=False,
                      scene_camera_eye=dict(x=1.87, y=0.88, z=-0.64),
                      width=700, height=700,
                      margin=dict(l=65, r=50, b=65, t=90)
    )
    
    fig.show()

## 3D Rendering (mayavi)

In [ ]:
def render_3D(CHM_clip: np.ndarray):
    '''
    This function uses mayavi to create an interactive 3D plot. 
    '''
    
    # Retrieves the NxM array from the BxNxM xarray object
    arr = CHM_clip.squeeze().data
    # Pads the array for better rendering
    arr = np.pad(arr, [(2, ), (2, )], mode='constant')
    mlab.figure(size=(640, 800), bgcolor=(0.16, 0.28, 0.46))
    surf = mlab.surf(arr)
    mlab.zlabel("Height")
    arg=f"""
    Max_height: {property["Max_height"]} m
    Estimated ground living area : {property['Estimated ground living area']} m2
    Estimated Max number of floors : {property['Estimated Max number of floors']}
    """
     
    ad=f'{dropdown_address.value}' 
    mlab.title(ad,color=(0.2, 1, 1),size=0.5)
    mlab.text(0,0.02,arg,color=(1, 1, 1), width=0.5)
    mlab.show()

## Widgets

In [ ]:
dropdown_address = widgets.Dropdown(
    options=['Sint-Pietersplein 9 Gent 9000', 
            'Sint-Pietersplein 16 Gent 9000', 
            'Lange Nieuwstraat 73 Antwerpen 2000',
            'Abrahamstraat 15 Gent 9000',
            'Quinten Matsijslei 25 Antwerpen 2018',
            'Koningin Astridplein 27 Antwerpen 2018'],
    value='Quinten Matsijslei 25 Antwerpen 2018',
    description='Address:',
    disabled=False,
)

button = widgets.Button(
    description='3D Plot',
    disabled=False,
    button_style='info', 
    tooltip='Click me',
    icon='cube'
)

dropdown_cut = widgets.Dropdown(
    options=[('Polygon clipping', True), ('Square clipping', False)],
    value=True,
    description='Clip:',
    disabled=False,
)

dropdown_renderer = widgets.Dropdown(
    options=[('Mayavi', render_3D), ('Plotly', render_3D_2)],
    value=render_3D_2,
    description='Renderer:',
    disabled=False,
)

def main(from_button):
    '''
    This function calls all the functions to process an address (within a widget)
    into a 3D plot and a dictionary with properties information.
    '''
    
    data = load_data(DSM_path, DTM_path)
    coord = fetch_coord(dropdown_address.value)
    tif_index = find_tif(data, coord)
    poly = find_polygon(coord, shp_paths)
    DSM_clip, DTM_clip = crop_tif(data, tif_index, poly, shape_cut=dropdown_cut.value)
    CHM = CHMer(DSM_clip, DTM_clip)
    property["Max_height"] = f"{round(np.max(CHM.data))}"
    property["Estimated ground living area"] = f"{int(poly.area * 0.7)}" # Roughly 70% of built-area is living area 
    property["Estimated Max number of floors"] = f"{int(np.max(CHM.data) / 3.5)} storey building"
    dropdown_renderer.value(CHM)
    
button.on_click(main)

items = [dropdown_address, dropdown_cut, dropdown_renderer, button]

## Showcasing MVP

In [ ]:
widgets.Box(items)